In [51]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error as MSE
import matplotlib.pyplot as plt

# データの読み込み
train = pd.read_csv('/content/drive/MyDrive/signate/練習 民泊サービスの宿泊価格予測/train.csv')
test = pd.read_csv('/content/drive/MyDrive/signate/練習 民泊サービスの宿泊価格予測/test.csv')
sample=pd.read_csv('/content/drive/MyDrive/signate/練習 民泊サービスの宿泊価格予測/sample_submit.csv',header=None)

In [52]:
train.shape

(55583, 29)

In [53]:
y=train['y']
train.drop(['y','id'], axis=1, inplace=True)
test.drop(['id'], axis=1, inplace=True)
all_data = pd.concat((train, test)).reset_index(drop=True)

In [54]:
all_data.isnull().sum()

accommodates                  0
amenities                     0
bathrooms                   200
bed_type                      0
bedrooms                     91
beds                        131
cancellation_policy           0
city                          0
cleaning_fee                  0
description                   0
first_review              15864
host_has_profile_pic        188
host_identity_verified      188
host_response_rate        18299
host_since                  188
instant_bookable              0
last_review               15827
latitude                      0
longitude                     0
name                          0
neighbourhood              6872
number_of_reviews             0
property_type                 0
review_scores_rating      16722
room_type                     0
thumbnail_url              8216
zipcode                     966
dtype: int64

In [55]:
# 年月関連の変数を全て浮動小数点型に変換し、欠損値は0で埋める
for c in ('first_review','last_review','host_since'):
    all_data[c] = pd.to_datetime(all_data[c])
    all_data[c] = pd.DatetimeIndex(all_data[c])
    all_data[c] = np.log(all_data[c].values.astype(np.float64))
    all_data[c] = all_data[c].fillna(0)

# 欠損値を1で埋める
for c in ('bathrooms','beds','bedrooms'):
    all_data[c] = all_data[c].fillna(1)

# 欠損値をNoneで埋める
for c in ('host_response_rate','neighbourhood','host_identity_verified','host_has_profile_pic'):
    all_data[c] = all_data[c].fillna('None')

# 中央値で埋める
all_data['review_scores_rating'].fillna(all_data['review_scores_rating'].median(),inplace=True)

# 使わない列を落とす
all_data = all_data.drop(['thumbnail_url','zipcode'],axis=1)


<ipython-input-55-8c1bbfed8085>:5: RuntimeWarning: invalid value encountered in log
  all_data[c] = np.log(all_data[c].values.astype(np.float64))
<ipython-input-55-8c1bbfed8085>:5: RuntimeWarning: invalid value encountered in log
  all_data[c] = np.log(all_data[c].values.astype(np.float64))
<ipython-input-55-8c1bbfed8085>:5: RuntimeWarning: invalid value encountered in log
  all_data[c] = np.log(all_data[c].values.astype(np.float64))


In [56]:
# ラベルエンコーディング
cols =  ('bed_type','cancellation_policy','city','cleaning_fee','host_identity_verified','host_has_profile_pic','host_response_rate','instant_bookable','property_type','room_type','neighbourhood')
from sklearn.preprocessing import LabelEncoder
for c in cols:
    lbl = LabelEncoder()
    lbl.fit(list(all_data[c].values))
    all_data[c] = lbl.transform(list(all_data[c].values))

# 対象の列の文字数をカウント
for c in ('amenities','description','name'):
    all_data[c] = all_data[c].apply(lambda x: sum(len(word) for word in str(x).split(" ")))

In [57]:
train =  all_data.head(55583)
test = all_data.iloc[55583:, :]

In [58]:
#目的変数及び説明変数を表す変数の準備
y_train = y # 目的変数
X_train = train
X_test =test

In [59]:
# モデルの箱の準備および学習
ridge = Ridge(alpha=0.1)
ridge.fit(X_train, y_train)
# 予測値の算出
y_pred_train=ridge.predict(X_train)
rmse=np.sqrt(MSE(y_pred_train,y_train))
print(rmse)
y_test = ridge.predict(X_test)
sample.iloc[:, 0].tail()

128.22949764598732


18523    18523
18524    18524
18525    18525
18526    18526
18527    18527
Name: 0, dtype: int64

In [60]:
y_test_series = pd.Series(y_test)
result_df = pd.concat([sample.iloc[:, 0],y_test_series], axis=1)
result_df.to_csv('/content/drive/MyDrive/signate/練習 民泊サービスの宿泊価格予測/submit.csv', header=False, index=False)